# LNG Physical Trade VaR Calculator (Parametric Approach)

This script calculates the Value at Risk (VaR) for a physical LNG cargo.

In [2]:
import numpy as np
from scipy.stats import norm

# --- Trade & Market Parameters ---
cargo_size_mmbtu = 3500000      # Standard cargo size (MMBtu)
current_price = 12.50           # USD/MMBtu (e.g., JKM Index)
volatility_ann = 0.40           # 40% annualized volatility
confidence_level = 0.95         # 95% confidence interval
holding_period_days = 5         # Time horizon (e.g., voyage/liquidation time)
trading_days_per_year = 252     # Standard trading days

# --- Calculations ---

# 1. Total Position Value
position_value = cargo_size_mmbtu * current_price

# 2. Scale Volatility to Holding Period
# Formula: Daily Vol * sqrt(Days)
sigma_hp = volatility_ann * np.sqrt(holding_period_days / trading_days_per_year)

# 3. Determine Z-Score
# For 95% confidence, z is approx 1.645
z_score = norm.ppf(confidence_level)

# 4. Calculate VaR
#the maximum loss you expect to see 95% of the time over 5 days
var_amount = position_value * sigma_hp * z_score 

# --- Results Output ---
print(f"LNG VaR ANALYSIS REPORT")
print("-" * 30)
print(f"Position Value: ${position_value:,.2f}")
print(f"Holding Period: {holding_period_days} days")
print(f"Confidence Level: {confidence_level:.0%}")
print(f"Expected Volatility: {sigma_hp:.2%}")
print("-" * 30)
print(f"VALUE AT RISK (VaR): ${var_amount:,.2f}")
print("-" * 30)
print(f"Interpretation: There is a {(1-confidence_level):.0%} chance that the ")
print(f"cargo will lose more than ${var_amount:,.2f} over the next {holding_period_days} days.")

LNG VaR ANALYSIS REPORT
------------------------------
Position Value: $43,750,000.00
Holding Period: 5 days
Confidence Level: 95%
Expected Volatility: 5.63%
------------------------------
VALUE AT RISK (VaR): $4,054,618.87
------------------------------
Interpretation: There is a 5% chance that the 
cargo will lose more than $4,054,618.87 over the next 5 days.


## Monte Carlo simulation

In [12]:
import numpy as np

# --- Parameters (From your previous script) ---
position_value = 43750000  # $43.75M
volatility_ann = 0.40
holding_period_days = 5
trading_days_per_year = 252
iterations = 10000         # Number of "what-if" scenarios

# --- 1. Calculate Periodic Volatility ---
# how much the price typically moves in a 5-day window so random numbers have range
sigma_hp = volatility_ann * np.sqrt(holding_period_days / trading_days_per_year)

# --- 2. Run the Simulation ---
# random seed ensures that each time code is run you get same random numbers. 
np.random.seed(42) # For consistent results
# We generate 10,000 random returns based on a normal distribution
# Using a 0% expected return ensures a conservative risk estimate
# not "banking" on profits which is what a <0% level would be
# Random Walk = Over short periods, price movememnts are noise, not steady trends. 
simulated_returns = np.random.normal(0, sigma_hp, iterations)

# --- 3. Calculate Simulated Gains/Losses ---
# We take those 10,000 random returns to our original total value
simulated_pnl = position_value * simulated_returns

# --- 4. Find the 95% VaR ---
# We take all 10,000 results and sort them from the worst loss to the biggest gain
# We then look at the 500th worst results
# Looking at 500th worst results is conservative approach, looking at worst cases so as to inform decisions
var_mc = np.percentile(simulated_pnl, 5)

# 95% of the time, our loss was better than this number
print(f"Monte Carlo VaR (95%): ${abs(var_mc):,.2f}")

Monte Carlo VaR (95%): $4,079,280.50


## TTF Futures model

1. Unconditional VaR, treats every 5-day period in the last two years as equally likely to happen, regardless of the season or current market events.

In [14]:
import pandas as pd
import numpy as np

# Use 'r' before the quotes to handle the Windows backslashes correctly
file_path = r'C:\Users\jongi\Documents\FInancial Management\Centrica\TTF NGF.csv'

# Load the data
df = pd.read_csv(file_path)

# Quick check to make sure it loaded
print("File loaded successfully!")
print(df.head()) # Shows the first 5 rows

File loaded successfully!
         Date   Price    Open    High     Low   Vol. Change %
0  26/01/2026  40.130  40.800  40.800  40.130  0.01K    0.23%
1  23/01/2026  40.039  37.200  40.039  37.200  0.29K    5.15%
2  22/01/2026  38.079  39.225  41.800  37.450  0.42K   -2.88%
3  21/01/2026  39.208  34.550  39.925  34.400  0.40K    9.25%
4  20/01/2026  35.887  34.700  37.095  33.215  1.12K    2.20%


In [46]:
import pandas as pd
import numpy as np

# --- 1. CONFIGURATION ---
path = 'TTF NGF.csv'
cargo_size_mmbtu = 3500000     
confidence_level = 95
holding_period = 5 

# --- 2. DATA LOAD & CLEANING ---
df = pd.read_csv(path)
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df = df.sort_values('Date')

# --- 3. VALUATION ---
# Using the most recent price from your Feb-26 contract
latest_price_mwh = df['Price'].iloc[-1]
latest_price_mmbtu = latest_price_mwh / 3.412
current_value = cargo_size_mmbtu * latest_price_mmbtu

# --- 4. HISTORICAL ANALYSIS ---
# Calculate rolling returns over the 5-day window
df['5d_Return'] = df['Price'].pct_change(holding_period)
clean_returns = df['5d_Return'].dropna()

# VaR Calculation (95th Percentile)
var_pct = np.percentile(clean_returns, 100 - confidence_level)
var_dollar = abs(var_pct * current_value)

# Stress Test: The absolute worst 5-day drop in the 2-year history
worst_case_pct = clean_returns.min()
worst_case_dollar = abs(worst_case_pct * current_value)

# --- 5. THE DEAL REPORT ---
print("="*65)
print(f"NG DEAL VaR REPORT: TTF FEB-26 FUTURES (TFAc1)")
print("="*65)

print(f"\n[DEAL SUMMARY]")
print(f"Asset:              Dutch TTF Natural Gas Feb '26")
print(f"Exposure:           {cargo_size_mmbtu:,.0f} MMBtu")
print(f"Market Price:       €{latest_price_mwh:.2f}/MWh (${latest_price_mmbtu:.2f}/MMBtu)")
print(f"Est Val of Cargo:   ${current_value:,.2f} USD")

print(f"\n[RISK METRICS - {holding_period} DAY WINDOW]")
print(f"Confidence Level:   {confidence_level}%")
# Rolling Volatility = the intensity or speed of the price swings over a full year
print(f"Rolling Volatility: {clean_returns.std() * np.sqrt(252/holding_period):.2%} (Annualized)")
print(f"Value at Risk (VaR): ${var_dollar:,.2f}")
print(f"VaR as % of Cargo:  {abs(var_pct):.2%}")

print(f"\n[STRESS TEST - 'THE SCAR']")
print(f"Worst 5-Day Move:   {worst_case_pct:.2%}")
print(f"Max Historical Loss: ${worst_case_dollar:,.2f}")

print("\n" + "="*65)
print(f"SUMMARY: There is a 95% probability that losses will not exceed")
print(f"${var_dollar:,.2f} over the next {holding_period} days.")
print(f"In a worst-case 'Stress' event, losses on the deal could reach ${worst_case_dollar:,.2f}.")
print("="*65)

NG DEAL VaR REPORT: TTF FEB-26 FUTURES (TFAc1)

[DEAL SUMMARY]
Asset:              Dutch TTF Natural Gas Feb '26
Exposure:           3,500,000 MMBtu
Market Price:       €40.13/MWh ($11.76/MMBtu)
Est Val of Cargo:   $41,165,005.86 USD

[RISK METRICS - 5 DAY WINDOW]
Confidence Level:   95%
Rolling Volatility: 48.13% (Annualized)
Value at Risk (VaR): $3,791,131.45
VaR as % of Cargo:  9.21%

[STRESS TEST - 'THE SCAR']
Worst 5-Day Move:   -19.16%
Max Historical Loss: $7,887,218.74

SUMMARY: There is a 95% probability that losses will not exceed
$3,791,131.45 over the next 5 days.
In a worst-case 'Stress' event, losses on the deal could reach $7,887,218.74.


## Targeted Window Historical Simulation

In [47]:
import pandas as pd
import numpy as np
from datetime import datetime

# --- 1. CONFIGURATION (Same as previous models) ---
path = r'C:\Users\jongi\Documents\FInancial Management\Centrica\TTF NGF.csv'
position_value_usd = 43750000      # Reverted to your exact deal value
confidence_level = 95
holding_period = 5 
ship_departure_date = "28/01/2026" # Today is 26th, ship leaves in 2 days
buffer_days = 30                  # Look at 15 days before/after Jan 28 in history

# --- 2. DATA LOAD & PREP ---
df = pd.read_csv(path)
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df = df.sort_values('Date')

# --- 3. THE "MICRO-SEASON" FILTER ---
# This isolates late-Jan/early-Feb data from 2024 and 2025
target_dt = datetime.strptime(ship_departure_date, "%d/%m/%Y")

def is_in_target_window(date):
    # Match the calendar window regardless of the year
    start_window = target_dt.replace(year=date.year) - pd.Timedelta(days=buffer_days)
    end_window = target_dt.replace(year=date.year) + pd.Timedelta(days=buffer_days)
    return start_window <= date <= end_window

df_relevant = df[df['Date'].apply(is_in_target_window)].copy()

# --- 4. CALCULATION ---
# Calculate 5-day returns specifically within these seasonal pockets
df_relevant['5d_Return'] = df_relevant['Price'].pct_change(holding_period)
clean_returns = df_relevant['5d_Return'].dropna()

# VaR Calculation
var_pct = np.percentile(clean_returns, 100 - confidence_level)
var_amount = abs(var_pct * position_value_usd)
worst_case_pct = clean_returns.min()
stress_loss = abs(worst_case_pct * position_value_usd)

# --- 5. UPDATED DEAL REPORT ---
print("="*65)
print(f"NG DEAL VaR REPORT: TARGETED DEPARTURE WINDOW")
print("="*65)

print(f"\n[DEAL SUMMARY]")
print(f"Risk Asset:         Dutch TTF Natural Gas Feb '26")
print(f"Ship Departure:     {ship_departure_date}")
print(f"Est Val of Cargo:   ${position_value_usd:,.2f} USD")

print(f"\n[RISK METRICS - {holding_period} DAY WINDOW]")
print(f"Confidence Level:   {confidence_level}%")
print(f"Relevant Samples:   {len(clean_returns)} days (Jan/Feb '24 & '25)")
print(f"Value at Risk (VaR): ${var_amount:,.2f}")
print(f"VaR as % of Cargo:  {abs(var_pct):.2%}")

print(f"\n[STRESS TEST - SEASONAL 'SCAR']")
print(f"Worst 5-Day Move:   {worst_case_pct:.2%}")
print(f"Max Historical Loss: ${stress_loss:,.2f}")

print("\n" + "="*65)
print(f"SUMMARY: This model isolates 'late-January' market behavior.")
print(f"It predicts a 95% likelihood that losses won't exceed ${var_amount:,.2f}")
print(f"during the 5-day voyage starting January 28th.")
print("="*65)

NG DEAL VaR REPORT: TARGETED DEPARTURE WINDOW

[DEAL SUMMARY]
Risk Asset:         Dutch TTF Natural Gas Feb '26
Ship Departure:     28/01/2026
Est Val of Cargo:   $43,750,000.00 USD

[RISK METRICS - 5 DAY WINDOW]
Confidence Level:   95%
Relevant Samples:   73 days (Jan/Feb '24 & '25)
Value at Risk (VaR): $14,383,819.38
VaR as % of Cargo:  32.88%

[STRESS TEST - SEASONAL 'SCAR']
Worst 5-Day Move:   -41.88%
Max Historical Loss: $18,323,514.13

SUMMARY: This model isolates 'late-January' market behavior.
It predicts a 95% likelihood that losses won't exceed $14,383,819.38
during the 5-day voyage starting January 28th.
